# Sales Data Analysis with Cassandra

This notebook demonstrates loading sales data into Cassandra using a medallion
architecture approach (bronze, silver, and gold layers).


## Setup and Data Loading

First, let's import required libraries and load our sales data


In [1]:
from cassandra.cluster import Cluster
import pandas as pd

file_path = "data/sales_100.csv"
sales_data = pd.read_csv(file_path)


## Connect to Cassandra and Create Keyspace


In [2]:
cluster = Cluster(["127.0.0.1"], port=9042, protocol_version=4, connect_timeout=60)
session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS sales_data
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
""")
session.set_keyspace("sales_data")


## Bronze Layer

Create and populate the bronze layer table with raw data


In [3]:
session.execute("""
CREATE TABLE IF NOT EXISTS bronze_sales (
    region text,
    country text,
    item_type text,
    sales_channel text,
    order_priority text,
    order_date text,
    order_id bigint PRIMARY KEY,  -- Changed to bigint to match integer format
    ship_date text,
    units_sold int,
    unit_price float,
    unit_cost float,
    total_revenue float,
    total_cost float,
    total_profit float
)
""")

for _, row in sales_data.iterrows():
    session.execute(
        """
    INSERT INTO bronze_sales (
        region, country, item_type, sales_channel, order_priority, 
        order_date, order_id, ship_date, units_sold, unit_price, 
        unit_cost, total_revenue, total_cost, total_profit
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
        (
            row["Region"],
            row["Country"],
            row["Item Type"],
            row["Sales Channel"],
            row["Order Priority"],
            row["Order Date"],
            row["Order ID"],
            row["Ship Date"],
            row["UnitsSold"],
            row["UnitPrice"],
            row["UnitCost"],
            row["TotalRevenue"],
            row["TotalCost"],
            row["TotalProfit"],
        ),
    )


## Silver Layer

Create and populate the silver layer with aggregated data by country and item
type


In [4]:
session.execute("""
CREATE TABLE IF NOT EXISTS silver_sales (
    country text,
    item_type text,
    sales_channel text,
    total_revenue float,
    total_profit float,
    PRIMARY KEY (country, item_type)
)
""")

silver_data = (
    sales_data.groupby(["Country", "Item Type", "Sales Channel"])
    .agg({"TotalRevenue": "sum", "TotalProfit": "sum"})
    .reset_index()
)

for _, row in silver_data.iterrows():
    session.execute(
        """
    INSERT INTO silver_sales (country, item_type, sales_channel, total_revenue, total_profit)
    VALUES (%s, %s, %s, %s, %s)
    """,
        (
            row["Country"],
            row["Item Type"],
            row["Sales Channel"],
            row["TotalRevenue"],
            row["TotalProfit"],
        ),
    )


## Gold Layer

Create and populate gold layer tables with specific business metrics


In [5]:
session.execute("""
CREATE TABLE IF NOT EXISTS gold_revenue_by_region (
    region text PRIMARY KEY,
    total_revenue float
)
""")
gold_data1 = sales_data.groupby("Region")["TotalRevenue"].sum().reset_index()
for _, row in gold_data1.iterrows():
    session.execute(
        """
    INSERT INTO gold_revenue_by_region (region, total_revenue)
    VALUES (%s, %s)
    """,
        (row["Region"], row["TotalRevenue"]),
    )

# Top Selling Items
session.execute("""
CREATE TABLE IF NOT EXISTS gold_top_selling_items (
    item_type text PRIMARY KEY,
    total_units_sold int
)
""")
gold_data2 = sales_data.groupby("Item Type")["UnitsSold"].sum().reset_index()
for _, row in gold_data2.iterrows():
    session.execute(
        """
    INSERT INTO gold_top_selling_items (item_type, total_units_sold)
    VALUES (%s, %s)
    """,
        (row["Item Type"], row["UnitsSold"]),
    )


session.execute("""
CREATE TABLE IF NOT EXISTS gold_high_priority_orders (
    order_id bigint PRIMARY KEY,
    order_priority text,
    total_profit float
)
""")
high_priority_orders = sales_data[sales_data["Order Priority"] == "H"]
for _, row in high_priority_orders.iterrows():
    session.execute(
        """
    INSERT INTO gold_high_priority_orders (order_id, order_priority, total_profit)
    VALUES (%s, %s, %s)
    """,
        (row["Order ID"], row["Order Priority"], row["TotalProfit"]),
    )


## View Results

Display sample data from each layer


In [6]:
print("\nSilver Table")
silver_data = session.execute("SELECT * FROM silver_sales").all()
for row in silver_data:
    print(row)



Silver Table
Row(country='Malaysia', item_type='Beverages', sales_channel='Online', total_profit=143351.640625, total_revenue=434357.3125)
Row(country='Israel', item_type='Beverages', sales_channel='Offline', total_profit=73742.9375, total_revenue=223442.046875)
Row(country='Serbia', item_type='Clothes', sales_channel='Offline', total_profit=539637.125, total_revenue=802989.4375)
Row(country='Djibouti', item_type='Clothes', sales_channel='Offline', total_profit=41273.28125, total_revenue=61415.359375)
Row(country='Egypt', item_type='Clothes', sales_channel='Online', total_profit=87540.4765625, total_revenue=130261.7578125)
Row(country='Nicaragua', item_type='Baby Food', sales_channel='Online', total_profit=277131.25, total_revenue=738014.5)
Row(country='Nicaragua', item_type='Household', sales_channel='Online', total_profit=1291202.375, total_revenue=5206491.5)
Row(country='Romania', item_type='Beverages', sales_channel='Online', total_profit=71738.4609375, total_revenue=217368.453125

In [7]:
print("\nBronze Table")
bronze_data = session.execute("SELECT * FROM bronze_sales LIMIT 10").all()
for row in bronze_data:
    print(row)



Bronze Table
Row(order_id=294530856, country='Italy', item_type='Cereal', order_date='11/15/2011', order_priority='M', region='Europe', sales_channel='Online', ship_date='12/28/2011', total_cost=829138.8125, total_profit=627217.1875, total_revenue=1456356.0, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=7080)
Row(order_id=274930989, country='Dominica', item_type='Household', order_date='11/19/2011', order_priority='C', region='Central America and the Caribbean', sales_channel='Offline', ship_date='12/13/2011', total_cost=3539891.75, total_profit=1167402.125, total_revenue=4707294.0, unit_cost=502.5400085449219, unit_price=668.27001953125, units_sold=7044)
Row(order_id=498071897, country='Taiwan', item_type='Cereal', order_date='4/11/2010', order_priority='H', region='Asia', sales_channel='Online', ship_date='5/26/2010', total_cost=1100482.625, total_profit=832480.25, total_revenue=1932962.875, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_s

In [8]:
print("\nGold Table: Top Selling Items")
gold_data2 = session.execute("SELECT * FROM gold_top_selling_items").all()
for row in gold_data2:
    print(row)



Gold Table: Top Selling Items
Row(item_type='Household', total_units_sold=57640)
Row(item_type='Office Supplies', total_units_sold=42814)
Row(item_type='Vegetables', total_units_sold=7368)
Row(item_type='Snacks', total_units_sold=14377)
Row(item_type='Personal Care', total_units_sold=39045)
Row(item_type='Meat', total_units_sold=50437)
Row(item_type='Fruits', total_units_sold=65920)
Row(item_type='Beverages', total_units_sold=45206)
Row(item_type='Cereal', total_units_sold=45776)
Row(item_type='Cosmetics', total_units_sold=65707)
Row(item_type='Baby Food', total_units_sold=20372)
Row(item_type='Clothes', total_units_sold=40148)


In [9]:
print("\nGold Table: Revenue by Region")
gold_data1 = session.execute("SELECT * FROM gold_revenue_by_region").all()
for row in gold_data1:
    print(row)



Gold Table: Revenue by Region
Row(region='Australia and Oceania', total_revenue=10711258.0)
Row(region='Europe', total_revenue=34964748.0)
Row(region='Middle East and North Africa', total_revenue=24765128.0)
Row(region='Central America and the Caribbean', total_revenue=17570836.0)
Row(region='Asia', total_revenue=28840812.0)
Row(region='Sub-Saharan Africa', total_revenue=24225438.0)
Row(region='North America', total_revenue=3611757.5)


In [10]:
print("\nGold Table: High Priority Orders")
gold_data3 = session.execute("SELECT * FROM gold_high_priority_orders").all()
for row in gold_data3:
    print(row)



Gold Table: High Priority Orders
Row(order_id=498071897, order_priority='H', total_profit=832480.25)
Row(order_id=368977391, order_priority='H', total_profit=1297765.625)
Row(order_id=222504317, order_priority='H', total_profit=70844.6171875)
Row(order_id=479969346, order_priority='H', total_profit=425981.5)
Row(order_id=835696351, order_priority='H', total_profit=1653322.5)
Row(order_id=371502530, order_priority='H', total_profit=73742.9375)
Row(order_id=146634709, order_priority='H', total_profit=126918.640625)
Row(order_id=179137074, order_priority='H', total_profit=543171.1875)
Row(order_id=529276502, order_priority='H', total_profit=55326.78125)
Row(order_id=887124383, order_priority='H', total_profit=768429.6875)
Row(order_id=824714744, order_priority='H', total_profit=26265.640625)
Row(order_id=797385394, order_priority='H', total_profit=367766.25)
Row(order_id=358570849, order_priority='H', total_profit=1380006.25)
Row(order_id=265081918, order_priority='H', total_profit=38915